Se tienen los Output de dos programas (Kraken y Kaiju), los cuales arrojan , c- Clasificado,  nombre del Read, y el Taxon; 

Objetivo: Hacer un scrpt que tome las salidas de varios programas  de asignacion taxonomica y los ponga en una tabla en la que : (Nombre de Read/ Taxon Asignado por Kraken/  Taxon Asignado por Kaiju) y así para lograr generalizar para varios programas.

1. Poner los datos de los outputs ordenados en la misma tabla...(Nombre/Kraken/Kaiju) 
    1. Leer un (.csv o .tsc) y convertirlo en diccionario.
    1. Hacer archivos mas pequeños de los datos
    1. Crear diccionarios para los datos
    1. Hacer un for para ordenar los datos



In [1]:
#lectura de los datos por medio de pandas, se tomaron las 10 primeras lineas de ambos outputs
import pandas as pd
#datakraken = pd.read_csv('data/EG1BMAA_kraken',sep='\t',header=None, names=["c-clasificado", "read","taxon","1","2"]) 
# pandas.leer archivo .csv(nombre del archivo, separado por tabulaciones, columnas sin nombre)
datakraken10 = pd.read_csv('data/10lineas_kraken',sep='\t',header=None, names=["c-clasificado", "read","taxon","1","2"])
#datakaiju = pd.read_csv('data/EG1BMAA_kaiju',sep='\t',header=None, names=["c-clasificado", "read","taxon"])
datakaiju10 = pd.read_csv('data/10lineas_kaiju',sep='\t',header=None, names=["c-clasificado", "read","taxon"])

In [2]:
datakraken10

,c-clasificado,read,taxon,1,2
0,C,NB501110:123:H7C3WBGX7:1:11101:9813:1065,547,151|151,A:6 2:5 1224:1 2:3 1236:10 91347:6 543:1 1236:...
1,C,NB501110:123:H7C3WBGX7:1:11101:9032:1067,329,150|150,A:6 119060:5 48736:5 329:13 0:4 329:7 0:6 1190...
2,C,NB501110:123:H7C3WBGX7:1:11101:9958:1067,69218,149|151,A:5 91347:17 69218:5 543:1 69218:21 543:4 6921...
3,C,NB501110:123:H7C3WBGX7:1:11101:22148:1069,1076549,150|150,A:6 1560356:1 91347:73 0:9 1560356:5 0:18 1560...
4,C,NB501110:123:H7C3WBGX7:1:11101:17061:1072,487316,150|151,A:6 469:57 487316:11 0:8 487316:2 0:8 469:3 0:...
5,C,NB501110:123:H7C3WBGX7:1:11101:7661:1075,69218,150|150,A:6 91347:3 547:5 91347:3 69218:14 547:1 69218...
6,C,NB501110:123:H7C3WBGX7:1:11101:3445:1076,1076549,151|151,A:6 91347:68 0:16 91347:2 0:12 91347:13 |:| 0:...
7,C,NB501110:123:H7C3WBGX7:1:11101:9350:1077,487316,151|150,A:6 0:3 469:64 487316:2 469:7 487316:22 469:13...
8,C,NB501110:123:H7C3WBGX7:1:11101:5326:1077,48736,151|151,A:6 2:111 |:| 48736:70 0:22 2:25
9,C,NB501110:123:H7C3WBGX7:1:11101:21410:1079,1560356,151|150,53335:11 91347:5 543:1 0:1 543:4 91347:3 2:6 5...


In [3]:
# como kraken10 tiene dos columnas extra que no necesitamos por el momento las eliminamos con .drop,
# asi quedan los dos outputs con las misma comunas
datakraken101 = datakraken10.drop(['c-clasificado','1', '2'], axis=1)
datakraken101

,read,taxon
0,NB501110:123:H7C3WBGX7:1:11101:9813:1065,547
1,NB501110:123:H7C3WBGX7:1:11101:9032:1067,329
2,NB501110:123:H7C3WBGX7:1:11101:9958:1067,69218
3,NB501110:123:H7C3WBGX7:1:11101:22148:1069,1076549
4,NB501110:123:H7C3WBGX7:1:11101:17061:1072,487316
5,NB501110:123:H7C3WBGX7:1:11101:7661:1075,69218
6,NB501110:123:H7C3WBGX7:1:11101:3445:1076,1076549
7,NB501110:123:H7C3WBGX7:1:11101:9350:1077,487316
8,NB501110:123:H7C3WBGX7:1:11101:5326:1077,48736
9,NB501110:123:H7C3WBGX7:1:11101:21410:1079,1560356


In [4]:
datakaiju101 = datakaiju10.drop(['c-clasificado'], axis=1)
datakaiju101

,read,taxon
0,NB501110:123:H7C3WBGX7:1:11101:22148:1069,470934
1,NB501110:123:H7C3WBGX7:1:11101:12474:1085,69218
2,NB501110:123:H7C3WBGX7:1:11101:19868:1087,53335
3,NB501110:123:H7C3WBGX7:1:11101:10733:1094,69218
4,NB501110:123:H7C3WBGX7:1:11101:20523:1097,1224136
5,NB501110:123:H7C3WBGX7:1:11101:9448:1118,470934
6,NB501110:123:H7C3WBGX7:1:11101:21936:1120,428406
7,NB501110:123:H7C3WBGX7:1:11101:25518:1127,329
8,NB501110:123:H7C3WBGX7:1:11101:22286:1152,1312959
9,NB501110:123:H7C3WBGX7:1:11101:6718:1180,53335


In [5]:
#Une los DataFrame, pero solo toma el total de los elementos del primero, por lo tanto se pierde informacion.
print (pd.merge(datakaiju101, datakraken101.rename(columns={'read':'read'}), on='read',  how='left'))

                                        read  taxon_x    taxon_y
0  NB501110:123:H7C3WBGX7:1:11101:22148:1069   470934  1076549.0
1  NB501110:123:H7C3WBGX7:1:11101:12474:1085    69218        NaN
2  NB501110:123:H7C3WBGX7:1:11101:19868:1087    53335        NaN
3  NB501110:123:H7C3WBGX7:1:11101:10733:1094    69218        NaN
4  NB501110:123:H7C3WBGX7:1:11101:20523:1097  1224136        NaN
5   NB501110:123:H7C3WBGX7:1:11101:9448:1118   470934        NaN
6  NB501110:123:H7C3WBGX7:1:11101:21936:1120   428406        NaN
7  NB501110:123:H7C3WBGX7:1:11101:25518:1127      329        NaN
8  NB501110:123:H7C3WBGX7:1:11101:22286:1152  1312959        NaN
9   NB501110:123:H7C3WBGX7:1:11101:6718:1180    53335        NaN


In [6]:
print (pd.merge(datakraken101, datakaiju101.rename(columns={'read':'read'}), on='read',  how='left'))

                                        read  taxon_x   taxon_y
0   NB501110:123:H7C3WBGX7:1:11101:9813:1065      547       NaN
1   NB501110:123:H7C3WBGX7:1:11101:9032:1067      329       NaN
2   NB501110:123:H7C3WBGX7:1:11101:9958:1067    69218       NaN
3  NB501110:123:H7C3WBGX7:1:11101:22148:1069  1076549  470934.0
4  NB501110:123:H7C3WBGX7:1:11101:17061:1072   487316       NaN
5   NB501110:123:H7C3WBGX7:1:11101:7661:1075    69218       NaN
6   NB501110:123:H7C3WBGX7:1:11101:3445:1076  1076549       NaN
7   NB501110:123:H7C3WBGX7:1:11101:9350:1077   487316       NaN
8   NB501110:123:H7C3WBGX7:1:11101:5326:1077    48736       NaN
9  NB501110:123:H7C3WBGX7:1:11101:21410:1079  1560356       NaN


In [7]:
datakraken101.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   read    10 non-null     object
 1   taxon   10 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes


In [8]:
datakaiju101.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   read    10 non-null     object
 1   taxon   10 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes


Cree dos diccionarios, cuyas 'keys' son los reads y los 'values' son los taxones segun la clacificación.

In [9]:
dickraken = datakraken101.set_index('read').T.to_dict('list')
dickraken

{'NB501110:123:H7C3WBGX7:1:11101:9813:1065': [547],
 'NB501110:123:H7C3WBGX7:1:11101:9032:1067': [329],
 'NB501110:123:H7C3WBGX7:1:11101:9958:1067': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:22148:1069': [1076549],
 'NB501110:123:H7C3WBGX7:1:11101:17061:1072': [487316],
 'NB501110:123:H7C3WBGX7:1:11101:7661:1075': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:3445:1076': [1076549],
 'NB501110:123:H7C3WBGX7:1:11101:9350:1077': [487316],
 'NB501110:123:H7C3WBGX7:1:11101:5326:1077': [48736],
 'NB501110:123:H7C3WBGX7:1:11101:21410:1079': [1560356]}

In [10]:
dickraken['NB501110:123:H7C3WBGX7:1:11101:22148:1069']

[1076549]

In [11]:
dickaiju = datakaiju101.set_index('read').T.to_dict('list')
dickaiju

{'NB501110:123:H7C3WBGX7:1:11101:22148:1069': [470934],
 'NB501110:123:H7C3WBGX7:1:11101:12474:1085': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:19868:1087': [53335],
 'NB501110:123:H7C3WBGX7:1:11101:10733:1094': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:20523:1097': [1224136],
 'NB501110:123:H7C3WBGX7:1:11101:9448:1118': [470934],
 'NB501110:123:H7C3WBGX7:1:11101:21936:1120': [428406],
 'NB501110:123:H7C3WBGX7:1:11101:25518:1127': [329],
 'NB501110:123:H7C3WBGX7:1:11101:22286:1152': [1312959],
 'NB501110:123:H7C3WBGX7:1:11101:6718:1180': [53335]}

In [26]:
dickaiju['NB501110:123:H7C3WBGX7:1:11101:22148:1069']

[470934]

In [27]:
#merge_dicts(dicKraken, dicKaiju)

In [28]:
#Este for crea un diccionario, tomando todos los datos de ambos diccionarios, mi mostrando el taxon de read para ambos outputs.
from itertools import chain
from collections import defaultdict
dic = defaultdict(list)
for k, v in chain(dickraken.items(), dickaiju.items()):
    dic[k].append(v)
dic 

defaultdict(list,
            {'NB501110:123:H7C3WBGX7:1:11101:9813:1065': [[547]],
             'NB501110:123:H7C3WBGX7:1:11101:9032:1067': [[329]],
             'NB501110:123:H7C3WBGX7:1:11101:9958:1067': [[69218]],
             'NB501110:123:H7C3WBGX7:1:11101:22148:1069': [[1076549],
              [470934]],
             'NB501110:123:H7C3WBGX7:1:11101:17061:1072': [[487316]],
             'NB501110:123:H7C3WBGX7:1:11101:7661:1075': [[69218]],
             'NB501110:123:H7C3WBGX7:1:11101:3445:1076': [[1076549]],
             'NB501110:123:H7C3WBGX7:1:11101:9350:1077': [[487316]],
             'NB501110:123:H7C3WBGX7:1:11101:5326:1077': [[48736]],
             'NB501110:123:H7C3WBGX7:1:11101:21410:1079': [[1560356]],
             'NB501110:123:H7C3WBGX7:1:11101:12474:1085': [[69218]],
             'NB501110:123:H7C3WBGX7:1:11101:19868:1087': [[53335]],
             'NB501110:123:H7C3WBGX7:1:11101:10733:1094': [[69218]],
             'NB501110:123:H7C3WBGX7:1:11101:20523:1097': [[1224

In [29]:
dic['NB501110:123:H7C3WBGX7:1:11101:22148:1069']

[[1076549], [470934]]

In [30]:
#Convertir el diccionario en un dataframe
data_items = dic.items()
data_list = list(data_items)

dfdic = pd.DataFrame(data_list)#, columns=['nombre_read', 'kaiju_taxon'])
dfdic

,0,1
0,NB501110:123:H7C3WBGX7:1:11101:9813:1065,[[547]]
1,NB501110:123:H7C3WBGX7:1:11101:9032:1067,[[329]]
2,NB501110:123:H7C3WBGX7:1:11101:9958:1067,[[69218]]
3,NB501110:123:H7C3WBGX7:1:11101:22148:1069,"[[1076549], [470934]]"
4,NB501110:123:H7C3WBGX7:1:11101:17061:1072,[[487316]]
5,NB501110:123:H7C3WBGX7:1:11101:7661:1075,[[69218]]
6,NB501110:123:H7C3WBGX7:1:11101:3445:1076,[[1076549]]
7,NB501110:123:H7C3WBGX7:1:11101:9350:1077,[[487316]]
8,NB501110:123:H7C3WBGX7:1:11101:5326:1077,[[48736]]
9,NB501110:123:H7C3WBGX7:1:11101:21410:1079,[[1560356]]


In [31]:
#df = pd.DataFrame(dic,columns=['nombre_read','kraken_taxon', 'kaiju_taxon'])
#df

In [32]:
#dfk = pd.DataFrame(dickaiju, index=['a','b','c'], columns=['nombre_read', 'kaiju_taxon'])
#dfk

In [58]:
dfk = pd.DataFrame(dickaiju)
dfk
#Esto toma los reads como columnas 

,NB501110:123:H7C3WBGX7:1:11101:22148:1069,NB501110:123:H7C3WBGX7:1:11101:12474:1085,NB501110:123:H7C3WBGX7:1:11101:19868:1087,NB501110:123:H7C3WBGX7:1:11101:10733:1094,NB501110:123:H7C3WBGX7:1:11101:20523:1097,NB501110:123:H7C3WBGX7:1:11101:9448:1118,NB501110:123:H7C3WBGX7:1:11101:21936:1120,NB501110:123:H7C3WBGX7:1:11101:25518:1127,NB501110:123:H7C3WBGX7:1:11101:22286:1152,NB501110:123:H7C3WBGX7:1:11101:6718:1180
0,470934,69218,53335,69218,1224136,470934,428406,329,1312959,53335


In [35]:
data_items = dickaiju.items()
data_list = list(data_items)

df = pd.DataFrame(data_list, columns=['nombre_read', 'kaiju_taxon'])
df

,nombre_read,kaiju_taxon
0,NB501110:123:H7C3WBGX7:1:11101:22148:1069,[470934]
1,NB501110:123:H7C3WBGX7:1:11101:12474:1085,[69218]
2,NB501110:123:H7C3WBGX7:1:11101:19868:1087,[53335]
3,NB501110:123:H7C3WBGX7:1:11101:10733:1094,[69218]
4,NB501110:123:H7C3WBGX7:1:11101:20523:1097,[1224136]
5,NB501110:123:H7C3WBGX7:1:11101:9448:1118,[470934]
6,NB501110:123:H7C3WBGX7:1:11101:21936:1120,[428406]
7,NB501110:123:H7C3WBGX7:1:11101:25518:1127,[329]
8,NB501110:123:H7C3WBGX7:1:11101:22286:1152,[1312959]
9,NB501110:123:H7C3WBGX7:1:11101:6718:1180,[53335]


In [36]:
data_items = dickraken.items()
data_list = list(data_items)

dfk = pd.DataFrame(data_list, columns=['nombre_read', 'kraken_taxon'])
dfk

,nombre_read,kraken_taxon
0,NB501110:123:H7C3WBGX7:1:11101:9813:1065,[547]
1,NB501110:123:H7C3WBGX7:1:11101:9032:1067,[329]
2,NB501110:123:H7C3WBGX7:1:11101:9958:1067,[69218]
3,NB501110:123:H7C3WBGX7:1:11101:22148:1069,[1076549]
4,NB501110:123:H7C3WBGX7:1:11101:17061:1072,[487316]
5,NB501110:123:H7C3WBGX7:1:11101:7661:1075,[69218]
6,NB501110:123:H7C3WBGX7:1:11101:3445:1076,[1076549]
7,NB501110:123:H7C3WBGX7:1:11101:9350:1077,[487316]
8,NB501110:123:H7C3WBGX7:1:11101:5326:1077,[48736]
9,NB501110:123:H7C3WBGX7:1:11101:21410:1079,[1560356]


In [39]:
#Une los DataFrame, pero solo toma el total de los elementos del primero, por lo tanto se pierde informacion.
print (pd.merge(df, dfk, left_on='nombre_read', right_on='nombre_read', how='left'))

                                 nombre_read kaiju_taxon kraken_taxon
0  NB501110:123:H7C3WBGX7:1:11101:22148:1069    [470934]    [1076549]
1  NB501110:123:H7C3WBGX7:1:11101:12474:1085     [69218]          NaN
2  NB501110:123:H7C3WBGX7:1:11101:19868:1087     [53335]          NaN
3  NB501110:123:H7C3WBGX7:1:11101:10733:1094     [69218]          NaN
4  NB501110:123:H7C3WBGX7:1:11101:20523:1097   [1224136]          NaN
5   NB501110:123:H7C3WBGX7:1:11101:9448:1118    [470934]          NaN
6  NB501110:123:H7C3WBGX7:1:11101:21936:1120    [428406]          NaN
7  NB501110:123:H7C3WBGX7:1:11101:25518:1127       [329]          NaN
8  NB501110:123:H7C3WBGX7:1:11101:22286:1152   [1312959]          NaN
9   NB501110:123:H7C3WBGX7:1:11101:6718:1180     [53335]          NaN


In [40]:
#Une dos diccionarios en uno solo, tomando todos los datos de ambos diccionarios
def mergeDict(dict1, dict2):
   #''' Merge dictionaries and keep values of common keys in list'''
   dic = {**dickraken, **dickaiju}
   for key, value in dic.items():
       if key in dickraken and key in dickaiju:
               dic[key] = [value , dickraken[key]]
   return dic
# Merge dictionaries and add values of common keys in a list
dic = mergeDict(dickraken, dickaiju)
dic

{'NB501110:123:H7C3WBGX7:1:11101:9813:1065': [547],
 'NB501110:123:H7C3WBGX7:1:11101:9032:1067': [329],
 'NB501110:123:H7C3WBGX7:1:11101:9958:1067': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:22148:1069': [[470934], [1076549]],
 'NB501110:123:H7C3WBGX7:1:11101:17061:1072': [487316],
 'NB501110:123:H7C3WBGX7:1:11101:7661:1075': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:3445:1076': [1076549],
 'NB501110:123:H7C3WBGX7:1:11101:9350:1077': [487316],
 'NB501110:123:H7C3WBGX7:1:11101:5326:1077': [48736],
 'NB501110:123:H7C3WBGX7:1:11101:21410:1079': [1560356],
 'NB501110:123:H7C3WBGX7:1:11101:12474:1085': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:19868:1087': [53335],
 'NB501110:123:H7C3WBGX7:1:11101:10733:1094': [69218],
 'NB501110:123:H7C3WBGX7:1:11101:20523:1097': [1224136],
 'NB501110:123:H7C3WBGX7:1:11101:9448:1118': [470934],
 'NB501110:123:H7C3WBGX7:1:11101:21936:1120': [428406],
 'NB501110:123:H7C3WBGX7:1:11101:25518:1127': [329],
 'NB501110:123:H7C3WBGX7:1:11101:22286:1152': [1312959]

In [43]:
data_items = dic.items()
data_list = list(data_items)
dfdic = pd.DataFrame(data_list)

In [44]:
keyskraken = dict.keys(dickraken)
keyskaiju = dict.keys(dickaiju)

In [45]:
keyskraken & keyskaiju

{'NB501110:123:H7C3WBGX7:1:11101:22148:1069'}

In [46]:
keyskraken - keyskaiju

{'NB501110:123:H7C3WBGX7:1:11101:17061:1072',
 'NB501110:123:H7C3WBGX7:1:11101:21410:1079',
 'NB501110:123:H7C3WBGX7:1:11101:3445:1076',
 'NB501110:123:H7C3WBGX7:1:11101:5326:1077',
 'NB501110:123:H7C3WBGX7:1:11101:7661:1075',
 'NB501110:123:H7C3WBGX7:1:11101:9032:1067',
 'NB501110:123:H7C3WBGX7:1:11101:9350:1077',
 'NB501110:123:H7C3WBGX7:1:11101:9813:1065',
 'NB501110:123:H7C3WBGX7:1:11101:9958:1067'}

In [47]:
keyskaiju - keyskraken 

{'NB501110:123:H7C3WBGX7:1:11101:10733:1094',
 'NB501110:123:H7C3WBGX7:1:11101:12474:1085',
 'NB501110:123:H7C3WBGX7:1:11101:19868:1087',
 'NB501110:123:H7C3WBGX7:1:11101:20523:1097',
 'NB501110:123:H7C3WBGX7:1:11101:21936:1120',
 'NB501110:123:H7C3WBGX7:1:11101:22286:1152',
 'NB501110:123:H7C3WBGX7:1:11101:25518:1127',
 'NB501110:123:H7C3WBGX7:1:11101:6718:1180',
 'NB501110:123:H7C3WBGX7:1:11101:9448:1118'}

In [51]:
def mergeDict(dict1, dict2):
   #'Merge dictionaries and keep values of common keys in list'
    keys1 = dict.keys(dict1)
    keys2 = dict.keys(dict2)
    interccion = keys1 & keys2
    dif1 = keys1 - keys2
    dif2 = keys2 - keys1
    dicc = {"read":[],"kraken_taxon":[],"kaiju_taxon":[],"measure":[]}
   #dic = {**dict1, **dict2}
    for i in interccion:
        dicc["read"].append(i)
        dicc["kraken_taxon"].append(dict1[i])
        dicc["kaiju_taxon"].append(dict2[i])
        if dict1[i] == dict2[i]:
            dicc["measure"].append(1)
        dicc["measure"].append(0)
    for i in dif1:
        dicc["read"].append(i)
        dicc["kraken_taxon"].append(dict1[i])
        dicc["kaiju_taxon"].append('NA')
        dicc["measure"].append(0)  
    for i in dif2:
        dicc["read"].append(i)
        dicc["kraken_taxon"].append('NA')
        dicc["kaiju_taxon"].append(dict2[i])
        dicc["measure"].append(0)  
    return dicc
# Merge dictionaries and add values of common keys in a list
dicc = mergeDict(dickraken, dickaiju)
dicc

{'read': ['NB501110:123:H7C3WBGX7:1:11101:22148:1069',
  'NB501110:123:H7C3WBGX7:1:11101:5326:1077',
  'NB501110:123:H7C3WBGX7:1:11101:17061:1072',
  'NB501110:123:H7C3WBGX7:1:11101:9813:1065',
  'NB501110:123:H7C3WBGX7:1:11101:9350:1077',
  'NB501110:123:H7C3WBGX7:1:11101:9958:1067',
  'NB501110:123:H7C3WBGX7:1:11101:21410:1079',
  'NB501110:123:H7C3WBGX7:1:11101:9032:1067',
  'NB501110:123:H7C3WBGX7:1:11101:7661:1075',
  'NB501110:123:H7C3WBGX7:1:11101:3445:1076',
  'NB501110:123:H7C3WBGX7:1:11101:10733:1094',
  'NB501110:123:H7C3WBGX7:1:11101:9448:1118',
  'NB501110:123:H7C3WBGX7:1:11101:21936:1120',
  'NB501110:123:H7C3WBGX7:1:11101:22286:1152',
  'NB501110:123:H7C3WBGX7:1:11101:20523:1097',
  'NB501110:123:H7C3WBGX7:1:11101:25518:1127',
  'NB501110:123:H7C3WBGX7:1:11101:12474:1085',
  'NB501110:123:H7C3WBGX7:1:11101:6718:1180',
  'NB501110:123:H7C3WBGX7:1:11101:19868:1087'],
 'kraken_taxon': [[1076549],
  [48736],
  [487316],
  [547],
  [487316],
  [69218],
  [1560356],
  [329],
 

In [63]:
#convierte el diccionario en data frame
dfdicc = pd.DataFrame(dicc)
dfdicc

,read,kraken_taxon,kaiju_taxon,measure
0,NB501110:123:H7C3WBGX7:1:11101:22148:1069,[1076549],[470934],0
1,NB501110:123:H7C3WBGX7:1:11101:5326:1077,[48736],NA,0
2,NB501110:123:H7C3WBGX7:1:11101:17061:1072,[487316],NA,0
3,NB501110:123:H7C3WBGX7:1:11101:9813:1065,[547],NA,0
4,NB501110:123:H7C3WBGX7:1:11101:9350:1077,[487316],NA,0
5,NB501110:123:H7C3WBGX7:1:11101:9958:1067,[69218],NA,0
6,NB501110:123:H7C3WBGX7:1:11101:21410:1079,[1560356],NA,0
7,NB501110:123:H7C3WBGX7:1:11101:9032:1067,[329],NA,0
8,NB501110:123:H7C3WBGX7:1:11101:7661:1075,[69218],NA,0
9,NB501110:123:H7C3WBGX7:1:11101:3445:1076,[1076549],NA,0
